# MPNN

## Import dependencies

In [ ]:
import os

import pandas as pd
import torch
from Bio.SeqIO.FastaIO import SimpleFastaParser
from tqdm.auto import tqdm

from pdmodels import MPNN
from pdmodels.globals import ROOT_DIR
from pdmodels.utils import get_dms_libary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = os.path.join(ROOT_DIR, "model_params", "ligandmpnn", "ligandmpnn_v_32_020_25.pt")
mpnn = MPNN(
    checkpoint_path,
    model_type="ligand_mpnn",
    device=device,
    ligand_mpnn_use_side_chain_context=True,
)

## Load data

In [ ]:
seqs_path = os.path.join(ROOT_DIR, "assets", "multimers", "multimers.fasta")
pdb_path = os.path.join(ROOT_DIR, "assets", "multimers", "NbALFA_ALFAtag_AF3.pdb")

with open(seqs_path) as f:
    seqs_list = [seq for _, seq in SimpleFastaParser(f)]

print(f"Multimer sequences list: {seqs_list}")

## Score sequences

In [ ]:
chains_to_design = "A"
redesigned_residues = "A1 A3 A4 A5 A7 A8 A9 A13 A14 A15 A19 A20 A21 A23 A24 A25 A26 A27 A39 A41 A44 A45 A46 A48 A50 A52 A53 A67 A68 A69 A72 A73 A74 A75 A76 A77 A78 A79 A80 A81 A82 A83 A84 A85 A86 A88 A89 A91 A92 A93 A95 A97 A99 A100 A102 A114 A116 A118 A119 A120 A121 A123 A124"
# symmetry_residues = "B1,B15|B5,B6,B7|B9,B10,B11"

output_dict = mpnn.score(
    pdb_path,
    # seqs_list=seqs_list,
    repeat=4,
    # chains_to_design=chains_to_design,
    redesigned_residues=redesigned_residues,
    # symmetry_residues=symmetry_residues,
    verbose=True,
)
output_dict

## DMS screen

In [ ]:
seqs_wt = "SGEVQLQESGGGLVQPGGSLRLSCTASGVTISALNAMAMGWYRQAPGERRVMVAAVSERGNAMYRESVQGRFTVTRDFTNKMVSLQMDNLKPEDTAVYYCHVLEDRVDSFHDYWGQGTQVTVSS:PSRLEEELRRRLTEP"
data = []
dms_library = get_dms_libary(seqs_wt)

batch_size = 32
for i in tqdm(range(0, len(dms_library), batch_size)):
    batch = dms_library[i : i + batch_size]
    batch_mutation_list, batch_seqs_list = zip(*batch)
    batch_perplexity_list = mpnn.score(
        pdb_path,
        seqs_list=batch_seqs_list,
        repeat=1,
        # chains_to_design=chains_to_design,
        redesigned_residues=redesigned_residues,
        # symmetry_residues=symmetry_residues,
        verbose=False,
    )["perplexity"].tolist()
    data.extend(list(zip(batch_mutation_list, batch_perplexity_list)))

df = pd.DataFrame(data, columns=["mutation", "perplexity"])
df